# PyTorch DDP Fashion MNIST Training Example run locally with Podman

This example demonstrates how to utilise Kubeflow Trainer locally with Podman. It simulates a similar experience to distributed training on kubernetes from your local machine. 

The notebook demonstrates how to train a convolutional neural network (CNN) to classify images using the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and [PyTorch Distributed Data Parallel (DDP)](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html). 


## Install the Kubeflow SDK

You need to install the Kubeflow SDK with the podman extra to interact with Kubeflow Trainer APIs:

In [ ]:
%pip install -U kubeflow[podman]

## Define the Training Function

The first step is to create function to train CNN model using Fashion MNIST data.

In [ ]:
def train_fashion_mnist():
    import os

    import torch
    import torch.distributed as dist
    import torch.nn.functional as F
    from torch import nn
    from torch.utils.data import DataLoader, DistributedSampler
    from torchvision import datasets, transforms

    # Define the PyTorch CNN model to be trained
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # Use NCCL if a GPU is available, otherwise use Gloo as communication backend.
    device, backend = ("cuda", "nccl") if torch.cuda.is_available() else ("cpu", "gloo")
    print(f"Using Device: {device}, Backend: {backend}")

    # Setup PyTorch distributed.
    local_rank = int(os.getenv("LOCAL_RANK", 0))
    dist.init_process_group(backend=backend)
    rank = dist.get_rank()
    print(
        "Distributed Training for WORLD_SIZE: {}, RANK: {}, LOCAL_RANK: {}".format(
            dist.get_world_size(),
            rank,
            local_rank,
        )
    )

    # Create the model and load it into the device.
    device = torch.device(f"{device}:{local_rank}")
    model = nn.parallel.DistributedDataParallel(Net().to(device))
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)


    # Use a rank-specific dataset directory to avoid concurrent writes to a shared mount
    data_dir = f"/tmp/fashion-mnist-{rank}"
    os.makedirs(data_dir, exist_ok=True)
    dataset = datasets.FashionMNIST(
        data_dir,
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()]),
    )


    # Shard the dataset accross workers.
    train_loader = DataLoader(
        dataset,
        batch_size=100,
        sampler=DistributedSampler(dataset)
    )

    # TODO(astefanutti): add parameters to the training function
    dist.barrier()
    for epoch in range(1, 3):
        model.train()

        # Iterate over mini-batches from the training set
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Copy the data to the GPU device if available
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(inputs)
            loss = F.nll_loss(outputs, labels)
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0 and dist.get_rank() == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(inputs),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    # Wait for the distributed training to complete
    dist.barrier()
    if dist.get_rank() == 0:
        print("Training is finished")

    # Finally clean up PyTorch distributed
    dist.destroy_process_group()

## Run PyTorch DDP with Kubeflow TrainJob

You can use `TrainerClient()` from the Kubeflow SDK to communicate with Kubeflow Trainer APIs and scale your training function across multiple PyTorch training nodes.

`TrainerClient(backend_config=ContainerBackendConfig())` verifies that you have required access to a local client.

Kubeflow Trainer creates a `TrainJob` resource and automatically sets the appropriate environment variables to set up PyTorch in a distributed environment. Distributed in this context means a local podman instance with multiple containers running communicating over a podman network.

In [ ]:
from kubeflow.trainer import CustomTrainer, TrainerClient, ContainerBackendConfig
import os

backend_config = ContainerBackendConfig()

# Or force use of Podman runtime:
# If you have both podman and docker installed it will use docker by default so you need to specify the backend_config you want to use.
# backend_config = ContainerBackendConfig(runtime="podman")
# If you need to specify a custom Podman socket URL, you can do so:
# backend_config = ContainerBackendConfig(
#     container_host="unix:///run/user/1000/podman/podman.sock"
# )
# Run the following command to set the podman socket URL:
# export CONTAINER_HOST="unix://$HOME/.local/share/containers/podman/machine/podman.sock"
# echo $CONTAINER_HOST to print the podman socket URL

client = TrainerClient(backend_config=backend_config)

## List the Training Runtimes

You can get the list of available Training Runtimes to start your TrainJob.

Additionally, it might show available accelerator type and number of available resources.

In [ ]:
for runtime in client.list_runtimes():
    print(runtime)
    if runtime.name == "torch-distributed":
        torch_runtime = runtime

## Run the Distributed TrainJob

Kubeflow TrainJob will train the above model on 2 PyTorch nodes.

In [ ]:
job_name = client.train(
    trainer=CustomTrainer(
        func=train_fashion_mnist,
        # Set how many PyTorch nodes you want to use for distributed training. 
        # num_nodes will equal the number of local containers running
        num_nodes=3, 
        packages_to_install=["torchvision"]
    ),
    runtime=torch_runtime,
)

## Check the TrainJob steps

You can check the components of TrainJob that's created.

Since the TrainJob performs distributed training across 3 nodes, it generates 3 steps: `trainer-node-0` .. `trainer-node-2`.

You can get the individual status for each of these steps.

In [ ]:
client.wait_for_job_status(name=job_name, status={"Running"})

In [ ]:
for c in client.get_job(name=job_name).steps:
    print(f"Step: {c.name}, Status: {c.status}, Devices: {c.device} x {c.device_count}\n")

## Watch the TrainJob logs

We can use the `get_job_logs()` API to get the TrainJob logs.

In [ ]:
for logline in client.get_job_logs(job_name, follow=True):
    print(logline)

## Optional: Examine Podman resources

- Containers for this training job

```bash
podman ps --filter label=trainer.kubeflow.ai/trainjob-name
```

Example:
```text
CONTAINER ID   IMAGE                                           NAMES
f6a786574f73   pytorch/pytorch:2.7.1-cuda12.8-cudnn9-runtime   ydb5bf3c10c4-node-1
c36274db6eb9   pytorch/pytorch:2.7.1-cuda12.8-cudnn9-runtime   ydb5bf3c10c4-node-0
```

- Network created for this training job

```bash
podman network ls --filter label=trainer.kubeflow.org/trainjob-name
```

Example:
```text
NETWORK ID     NAME               DRIVER    SCOPE
2cded187f9e7   b69f13d3f8dc-net   bridge    local
```









## Delete the TrainJob

When TrainJob is finished, you can delete the resource.

In [ ]:
client.delete_job(job_name)